## Construccion de un VP-tree

### Paso 1
Se selecciona un punto pivote de máxima diversidad, el pivote va a variar dependiendo del algoritmo implementado, en este caso estamos usando una aproximación en la que se consigue una muestra aleatoria $m$ de la cantidad de puntos $n$ tal que $m <<< n$

por lo que la complejidad esperada de este paso sería $O(nm)$ y dado que $m$ se considera constante entonces la complejidad tiende a $O(n)$

### Paso 2

Luego de determinado el pivote se procede a hacer la division de los datos uno a uno lo que tiene complejidad $O(n)$

### Paso 3
Se repite recursivamente para el arbol de la derecha como de la izquierda teniendo complejidad esperada de $O(log(n))$


En total la construcción del árbol tiene complejidad $O(nlog(n))$ cuando $m$ es constante

In [1]:
import heapq
import numpy as np
def euclidean_distance(a, b):
	return np.linalg.norm(a - b)

## Búsqueda de k vecinos cercanos

Para esto se usa la siguiente función:


In [2]:
def search_knn(target, node, k, heap):
    if node is None:
        return

    distance = euclidean_distance(target, node.point)
    
    if len(heap) < k:
        heapq.heappush(heap, (-distance, node.point))
    else:
        if distance < -heap[0][0]:
            heapq.heappushpop(heap, (-distance, node.point))

    if distance < node.threshold:
        search_knn(target, node.left, k, heap)
        if (distance + (-heap[0][0])) >= node.threshold:  
            search_knn(target, node.right, k, heap)
    else:
        search_knn(target, node.right, k, heap)
        if (distance - (-heap[0][0])) <= node.threshold:
            search_knn(target, node.left, k, heap)

### Propiedades de la distancia en un espacio métrico
- $d(x,x) = 0$
- Si $x \neq y$ entonces $d(x,y) > 0$
- $d(x,y) = d(y,x)$
- $d(x,z) \leq d(x,y) + d(y,z)$

### ¿Cómo funciona la poda?

Para esto se usa la desigualdad triangular. Sea:
- target: El nodo al cual se le buscan los k vecinos más cercanos
- pivot: El punto el cual se va a comparar con target
- p: Punto de referencia con respecto a pivot 
- wd: Vecino obtenido con mayor distancia
$$
dist(p, target) \geq |dist(p, pivot) - dist(pivot, target)|        ...(1)
$$

Para el caso del target en el subárbol cercano se tiene:
- $dist(pivot, target) < threshold$
- $threshold < dist(p, pivot)$

Entonces $dist(pivot, target) < threshold < dist(p, pivot)$

Luego, (1) se convierte en:
$$
dist(p, target) \geq dist(p, pivot) - dist(pivot, target) 
$$

$$
dist(p, target) \geq dist(p, pivot) - dist(pivot, target) \geq threshold - distance 
$$

Entonces, si $threshold - distance \geq wd$, no puede haber un punto en el subárbol lejano que pueda entrar en el grupo de k vecinos.

Para el caso del target en el subárbol lejano se tiene:
- $dist(pivot, target) \geq threshold$
- $threshold > dist(p, pivot)$

Luego, (1) se convierte en:
$$
dist(p, target) \geq dist(pivot, target) - dist(p, pivot)  
$$

$$
dist(p, target) \geq dist(pivot, target) - dist(p, pivot) \geq distance - threshold 
$$




## Búsqueda en una región de radio k

Para esto se usa la siguiente función:

In [3]:
def region_search(node, target, radius, results):
    if node is None:
        return

    d = euclidean_distance(target, node.point)
    if d <= radius:
        results.append(node.point)

    if d < node.threshold:
        region_search(node.left, target, radius, results)
        if d + radius >= node.threshold:
            region_search(node.right, target, radius, results)
    else:
        region_search(node.right, target, radius, results)
        if d - radius <= node.threshold:
            region_search(node.left, target, radius, results)



# Cosas a tener en cuenta más adelante:
$$
d(x, z) \leq d(x, y) + d(y, z)
$$

implica que:

$$
|d(x, y) - d(y, z)| \leq d(x, z)
$$


$$
d(x, z) \leq d(x, y) + d(y, z) \tag{1}
$$


$$
d(x, y) \leq d(x, z) + d(z, y) \tag{2}
$$

Como $d(z, y) = d(y, z)$:

$$
d(x, y) \leq d(x, z) + d(y, z)
\Rightarrow d(x, y) - d(y, z) \leq d(x, z) \tag{3}
$$


$$
d(y, z) \leq d(y, x) + d(x, z)
\Rightarrow d(y, z) - d(x, y) \leq d(x, z) \tag{4}
$$


De (3):

$$
d(x, y) - d(y, z) \leq d(x, z)
$$

De (4):

$$
d(y, z) - d(x, y) \leq d(x, z)
$$

Entonces:

$$
|d(x, y) - d(y, z)| \leq d(x, z)
$$


### Target en subarbol cercano
La distancia de target a un punto p en el subárbol lejano cumple:

dist(p, target) ≥ |dist(p, pivot) - dist(pivot, target)| ≥ threshold - distance  
Si threshold - distance > worst_dist (o equivalentemente, distance + worst_dist < threshold), ningún p en el lejano puede estar más cerca que worst_dist.

### Target en subárbol lejano

La distancia de target a un punto p en el subárbol cercano cumple:

dist(p, target) ≥ dist(pivot, target) - dist(p, pivot) ≥ distance - threshold  
Si distance - threshold > worst_dist (o distance - worst_dist > threshold), ningún p en el cercano puede mejorar el heap.


